### 目标：将flower_photos图片封装层TFRecord文件，并读取TFRecord
[点击下载flower_photos数据集](http://download.tensorflow.org/example_images/flower_photos.tgz)

page 172

In [3]:
#图片转TFRecord文件

import tensorflow as tf
import glob,os
from PIL import Image

BATCH_SIZE=5
FLOWER_PHOTOS='flower_photos'
IS_TEST=True
OUTPUT=os.path.join(FLOWER_PHOTOS,'output.tfrecords')
IMAGE_SIZE=299

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def to_tfexample(image,label):
    return tf.train.Example(features=tf.train.Features(feature={
        'image':bytes_feature(image),
        'label':int64_feature(label)
    }))

def generate_tfrecord(writer):
    sub_dirs=[_[0] for _ in os.walk(FLOWER_PHOTOS)][1:]
    sub_dirs.remove('flower_photos/.ipynb_checkpoints')
    
    for index,_ in enumerate(sub_dirs):
        print 'label:%d, flower name: %s' %(index,os.path.basename(_))
        
        #拼接glob匹配的文件名
        re=os.path.join(_,'*.jpg')
        files=glob.glob(re)[:10] if IS_TEST else glob.glob(re)
        
        for path in files:
    
            image=Image.open(path)
            resized=image.resize((IMAGE_SIZE,IMAGE_SIZE))
            image_bytes=resized.tobytes()
            
            example=to_tfexample(image_bytes,index)
            writer.write(example.SerializeToString())

    
with tf.python_io.TFRecordWriter(OUTPUT) as writer:
    generate_tfrecord(writer)
    

label:0, flower name: roses
label:1, flower name: sunflowers
label:2, flower name: daisy
label:3, flower name: dandelion
label:4, flower name: tulips


In [ ]:
#读取TFRecord文件

def read_tfrecord(path=OUTPUT):
    reader=tf.TFRecordReader()
    filename_queue=tf.train.string_input_producer([path])
    
    _,serialized_example=reader.read(filename_queue)
    
    #解析example
    features=tf.parse_single_example(serialized_example,features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.int64)
    })
    single_image=tf.decode_raw(features['image'],tf.uint8)
    single_image=tf.reshape(single_image,[IMAGE_SIZE,IMAGE_SIZE,3])
    
    single_label=tf.cast(features['label'],tf.int32)

    return single_image,single_label

single_image,single_label=read_tfrecord()
image_batch, label_batch = tf.train.shuffle_batch(
        [single_image, single_label],
        batch_size=BATCH_SIZE,
        num_threads=4,
        capacity=50000,
        min_after_dequeue=10000
)

with tf.Session() as sess:
    # 创建一个协调器，管理线程
    coord = tf.train.Coordinator()
    # 启动QueueRunner, 此时文件名队列已经进队
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    #获取5批次数据
    for _ in range(4):
        _image_batch,_label_batch = sess.run([image_batch,label_batch])
        print _image_batch.shape,_label_batch
    
    
    
    # 通知其他线程关闭
    coord.request_stop()
    # 其他所有线程关闭之后，这一函数才能返回
    coord.join(threads)

(5, 299, 299, 3) [1 0 4 2 3]
(5, 299, 299, 3) [3 4 4 3 1]
(5, 299, 299, 3) [4 1 2 3 3]
(5, 299, 299, 3) [4 4 2 3 1]